In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import time
import random
import os
from math import *

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as MAE

In [ ]:
path = os.getcwd()
print("Le répertoire courant est : " + path)

Le répertoire courant est : /home/samih/Bureau/PROJET_AIF/Defi-IA-2023-main/Models


In [50]:
d= pd.read_csv('c.csv')
f= pd.read_csv('f.csv', index_col=['hotel_id' , 'city'])
#f= pd.read_csv('f.csv')

In [28]:
f.head(10)

,hotel_id,group,brand,city,parking,pool,children_policy
0,0,Yin Yang,Royal Lotus,paris,1,0,1
1,1,Independant,Independant,copenhagen,1,0,0
2,2,Independant,Independant,madrid,0,0,0
3,3,Independant,Independant,rome,1,0,0
4,4,Independant,Independant,sofia,1,0,0
5,5,Yin Yang,Royal Lotus,vilnius,0,1,0
6,6,Independant,Independant,copenhagen,1,0,0
7,7,Independant,Independant,madrid,0,0,0
8,8,Boss Western,Boss Western,vienna,1,0,0
9,9,Independant,Independant,amsterdam,0,0,0


In [ ]:
d.head(10)

,Unnamed: 0.1,Unnamed: 0,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,0,0,144,82,0,rome,0,cypriot,0,110364
1,1,1,651,116,0,rome,0,cypriot,0,110364
2,2,2,453,392,0,rome,0,cypriot,0,110364
3,3,3,583,74,0,rome,0,cypriot,0,110364
4,4,4,361,165,0,rome,0,cypriot,0,110364
5,5,5,42,259,0,rome,0,cypriot,0,110364
6,6,6,864,167,0,rome,0,cypriot,0,110364
7,7,7,206,178,5,rome,0,cypriot,0,110364
8,8,8,907,144,0,rome,0,cypriot,0,110364
9,9,9,874,280,0,rome,0,cypriot,0,110364


In [51]:
d= d.drop(d.columns[[0]], axis=1)

In [52]:
d_plus= d.join(f, on=['hotel_id', 'city'])

In [54]:
d_plus

,Unnamed: 0.1,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,0,144,82,0,rome,0,cypriot,0,110364,Yin Yang,8 Premium,0,0,0
1,1,651,116,0,rome,0,cypriot,0,110364,Independant,Independant,0,0,0
2,2,453,392,0,rome,0,cypriot,0,110364,Chillton Worldwide,Chill Garden Inn,0,0,2
3,3,583,74,0,rome,0,cypriot,0,110364,Accar Hotels,Ibas,0,0,0
4,4,361,165,0,rome,0,cypriot,0,110364,Boss Western,J.Halliday Inn,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422399,139,855,267,6,copenhagen,7,danish,0,212029,Yin Yang,Royal Lotus,0,0,0
422400,140,830,95,4,copenhagen,7,danish,0,212029,Independant,Independant,0,0,0
422401,141,92,89,8,copenhagen,7,danish,0,212029,Independant,Independant,0,0,0
422402,142,435,493,7,copenhagen,7,danish,0,212029,Chillton Worldwide,Chill Garden Inn,0,1,0


In [55]:
prix= d_plus["price"]

In [56]:
d_plus= d_plus.drop(['Unnamed: 0.1'], axis=1)
d_plus= d_plus.drop(['price'], axis=1)

In [57]:
d_plus_dummies= pd.get_dummies(d_plus)

In [58]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

In [59]:
#fonction des erreurs
def weighted_average(prev,test):
    abs = mean_absolute_error(test, prev)
    rel = np.mean(np.abs(np.array(test)-np.array(prev))/np.array(test))
    return abs, rel
def erreur(test,prev): 
    MSE= round(mean_squared_error(test,prev),2)
    MAE= round(mean_absolute_error(test,prev),2)
    MAPE= round(weighted_average(prev,test)[1],3) *100
    return MSE,MAE,MAPE

In [60]:
from xgboost import XGBRegressor
regressor = XGBRegressor(learning_rate=0.01,  
                      colsample_bytree = 1,
                      subsample = 1,
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=10, 
                      gamma=0)
regressor.fit(d_plus_dummies,prix)

[17:21:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(learning_rate=0.01, max_depth=10, n_estimators=1000, reg_alpha=0.3)

In [64]:
regressor.save_model('0001.model')

In [61]:
test_set= pd.read_csv('test_set.csv')

In [62]:
test_set_joined= test_set.join(f, on=['hotel_id', 'city'])

In [63]:
test_set_joined

,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
0,0,1,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,1,1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,2,1,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,3,1,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,4,1,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,6639,843,rome,5,irish,0,794,987,1,Accar Hotels,Ibas,1,0,0
6640,6640,844,vienna,1,irish,1,794,26,1,Accar Hotels,Marcure,1,0,0
6641,6641,844,vienna,1,irish,1,794,263,0,Boss Western,Boss Western,1,0,0
6642,6642,844,vienna,1,irish,1,794,456,0,Yin Yang,Ardisson,1,0,0


In [65]:
X_test = test_set_joined[['hotel_id', 'stock', 'city', 'date', 'language', 'mobile', 'avatar_id', 'group', 'brand', 'parking', 'pool', 'children_policy']]

In [66]:
X_test_dummies= pd.get_dummies(X_test)

In [96]:
pred_ayoub= regressor.predict(X_test_dummies)
pred_ayoub= pd.Series(pred_ayoub)

In [101]:
index= test_set_joined['index']
submission= pd.concat([index, pred_ayoub], axis=1)
submission.columns =['index', 'price']

In [102]:
submission

,index,price
0,0,109.020966
1,1,138.662216
2,2,59.439518
3,3,60.233711
4,4,212.319839
...,...,...
6639,6639,82.040520
6640,6640,224.541122
6641,6641,143.199570
6642,6642,172.109497


In [103]:
submission.to_csv('ayoub4.csv', index=False)

In [ ]:
# Prédiction sur l'échantillon test
#roro = regressor.predict(X_test_dummies)

#MSE_Rf,MAE_Rf,MAPE_Rf = erreur(final_test,roro)
#print("MSE=",MSE_Rf)
#print("MAE=",MAE_Rf)
#print('MAPE_Rf:', MAPE_Rf)

MSE= 68.82
MAE= 5.61
MAPE_Rf: 4.1000000000000005
